# Import Library

In [3]:
import pandas as pd
import numpy as np
import random
import datetime
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Data prepare.

## Import cc data

In [4]:
cc = pd.read_csv('../data_pack/cc.csv')
cc['pos_dt'] = pd.to_datetime(cc['pos_dt']) # convert to datetime formatted.
cc.head()

,cc_no,pos_dt,cc_txn_amt
0,37069,2018-05-10,5000
1,37069,2018-06-04,12000
2,37069,2018-04-03,5000
3,37069,2018-04-22,1600
4,37069,2018-01-21,5000


### Add column week of month. [ cc data ]

In [5]:
week_of_month = [int(i.strftime("%V")) for i in cc['pos_dt']]
cc['week_of_month'] = week_of_month
cc.head(10)

,cc_no,pos_dt,cc_txn_amt,week_of_month
0,37069,2018-05-10,5000,19
1,37069,2018-06-04,12000,23
2,37069,2018-04-03,5000,14
3,37069,2018-04-22,1600,16
4,37069,2018-01-21,5000,3
5,37201,2018-04-13,400,15
6,37201,2018-04-19,300,16
7,37201,2018-01-15,600,3
8,37201,2018-06-13,1500,24
9,37201,2018-01-22,1300,4


### Create new dataframe that group by cc_no and week_of_month.

In [6]:
cc_wom = cc.groupby(['cc_no', 'week_of_month']).sum()
cc_wom = cc_wom.reset_index()
cc_wom.head()

,cc_no,week_of_month,cc_txn_amt
0,2,10,800
1,2,11,3800
2,2,17,18700
3,2,19,800
4,2,20,800


In [7]:
print(sorted(cc_wom.week_of_month.value_counts().index)) # number of week

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]


## Import kplus data

In [8]:
kplus = pd.read_csv('../data_pack/kplus.csv')
kplus['sunday'] = pd.to_datetime(kplus['sunday']) # convert to datetime formatted.
kplus.head(10)

,id,sunday,kp_txn_count,kp_txn_amt
0,14802,2018-01-14,2,2400
1,14802,2018-04-01,9,33900
2,14802,2018-05-27,7,6100
3,14802,2018-01-28,3,8500
4,14802,2018-03-25,2,4000
5,14802,2018-05-06,12,35300
6,14802,2018-03-11,1,4500
7,14802,2018-01-07,6,20800
8,14802,2018-06-03,11,42900
9,14802,2018-02-04,11,49800


### Add column week of month. [ kplus data ]

In [9]:
kplus_week_of_month = [int(i.strftime("%V")) for i in kplus['sunday']]
kplus['week_of_month'] = kplus_week_of_month # add column week_of_month.
kplus.head(10)

,id,sunday,kp_txn_count,kp_txn_amt,week_of_month
0,14802,2018-01-14,2,2400,2
1,14802,2018-04-01,9,33900,13
2,14802,2018-05-27,7,6100,21
3,14802,2018-01-28,3,8500,4
4,14802,2018-03-25,2,4000,12
5,14802,2018-05-06,12,35300,18
6,14802,2018-03-11,1,4500,10
7,14802,2018-01-07,6,20800,1
8,14802,2018-06-03,11,42900,22
9,14802,2018-02-04,11,49800,5


In [10]:
print(sorted(kplus.week_of_month.value_counts().index)) # number of week

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


## Import demographics data.

In [11]:
demo = pd.read_csv('../data_pack/demographics.csv')
demo = demo.fillna(value={'ocp_cd':random.choice([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 11.0, 12.0, 13.0])})
demo = demo.interpolate(method ='nearest', limit_direction ='forward')
demo.head(10)

,id,cc_no,gender,ocp_cd,age
0,1,1,2,9.0,5
1,1,98397,2,9.0,5
2,2,2,2,3.0,4
3,2,9740,2,3.0,4
4,3,3,2,1.0,3
5,4,4,2,3.0,5
6,4,86813,2,3.0,5
7,5,5,2,9.0,4
8,6,6,1,3.0,3
9,6,91379,1,3.0,3


In [12]:
print(sorted(demo.ocp_cd.value_counts().index))

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 11.0, 12.0, 13.0]


## Mapping demographics with cc data.

In [13]:
# Prepare demographics data
demo_cc = demo.sort_values('cc_no')
demo_cc = demo_cc.set_index('cc_no')

# Show demographics dataframe
demo_cc.head(10)

,id,gender,ocp_cd,age
cc_no,,,,
1,1,2,9.0,5
2,2,2,3.0,4
3,3,2,1.0,3
4,4,2,3.0,5
5,5,2,9.0,4
6,6,1,3.0,3
7,7,1,3.0,5
8,8,2,9.0,3
9,9,1,9.0,4


In [14]:
# show cc dataframe.
cc_wom.head(10)

,cc_no,week_of_month,cc_txn_amt
0,2,10,800
1,2,11,3800
2,2,17,18700
3,2,19,800
4,2,20,800
5,2,23,1000
6,2,24,1000
7,4,19,50000
8,4,20,100000
9,4,21,150000


In [15]:
# Mapping with cc data

for i in range(1,27):
    tmp = cc_wom[cc_wom['week_of_month'] == i]
    tmp = tmp.set_index('cc_no')
    demo_cc['cc_week_'+str(i)] = tmp.cc_txn_amt

#### Aggregate expense method

In [16]:
# keep original ocp_cd and gender before sum expense each week.
actual_value = demo_cc.groupby('id').mean()

In [17]:
# sum expense each week.
demo_f = demo_cc.groupby('id').sum()
demo_f

,gender,ocp_cd,age,cc_week_1,cc_week_2,cc_week_3,cc_week_4,cc_week_5,cc_week_6,cc_week_7,...,cc_week_17,cc_week_18,cc_week_19,cc_week_20,cc_week_21,cc_week_22,cc_week_23,cc_week_24,cc_week_25,cc_week_26
id,,,,,,,,,,,,,,,,,,,,,
1,4,18.0,10,0.0,0.0,4700.0,0.0,0.0,0.0,20000.0,...,0.0,0.0,10000.0,0.0,0.0,0.0,0.0,10000.0,0.0,0.0
2,4,6.0,8,1600.0,0.0,0.0,3500.0,2600.0,800.0,0.0,...,20500.0,3300.0,800.0,800.0,0.0,3500.0,2100.0,1000.0,0.0,5600.0
3,2,1.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,6.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,50000.0,100000.0,150000.0,0.0,0.0,0.0,140000.0,100000.0
5,2,9.0,4,0.0,0.0,5000.0,2000.0,0.0,5000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64996,2,3.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64997,2,3.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64998,1,9.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# replace [gender, ocp_cd, age] column with actual value.
demo_f.gender = actual_value.gender
demo_f.ocp_cd = actual_value.ocp_cd
demo_f.age = actual_value.age
demo_f

,gender,ocp_cd,age,cc_week_1,cc_week_2,cc_week_3,cc_week_4,cc_week_5,cc_week_6,cc_week_7,...,cc_week_17,cc_week_18,cc_week_19,cc_week_20,cc_week_21,cc_week_22,cc_week_23,cc_week_24,cc_week_25,cc_week_26
id,,,,,,,,,,,,,,,,,,,,,
1,2,9.0,5,0.0,0.0,4700.0,0.0,0.0,0.0,20000.0,...,0.0,0.0,10000.0,0.0,0.0,0.0,0.0,10000.0,0.0,0.0
2,2,3.0,4,1600.0,0.0,0.0,3500.0,2600.0,800.0,0.0,...,20500.0,3300.0,800.0,800.0,0.0,3500.0,2100.0,1000.0,0.0,5600.0
3,2,1.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,3.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,50000.0,100000.0,150000.0,0.0,0.0,0.0,140000.0,100000.0
5,2,9.0,4,0.0,0.0,5000.0,2000.0,0.0,5000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64996,2,3.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64997,2,3.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64998,1,9.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Mapping with kplus data

for i in range(1,26):
    tmp = kplus[kplus['week_of_month'] == i]
    tmp = tmp.set_index('id')
    demo_f['kp_week_'+str(i)] = tmp.kp_txn_amt

In [20]:
demo_f = demo_f.fillna(0)
demo_f

,gender,ocp_cd,age,cc_week_1,cc_week_2,cc_week_3,cc_week_4,cc_week_5,cc_week_6,cc_week_7,...,kp_week_16,kp_week_17,kp_week_18,kp_week_19,kp_week_20,kp_week_21,kp_week_22,kp_week_23,kp_week_24,kp_week_25
id,,,,,,,,,,,,,,,,,,,,,
1,2,9.0,5,0.0,0.0,4700.0,0.0,0.0,0.0,20000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,0.0,2000.0,0.0
2,2,3.0,4,1600.0,0.0,0.0,3500.0,2600.0,800.0,0.0,...,4200.0,26900.0,20600.0,1500.0,1100.0,1700.0,27700.0,1600.0,1600.0,40300.0
3,2,1.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5000.0,17400.0,300.0,10200.0,1900.0,4000.0,19900.0,100.0,200.0,2500.0
4,2,3.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,678400.0,217700.0,430300.0,87900.0,158400.0,52400.0,290400.0,500200.0,225200.0,394800.0
5,2,9.0,4,0.0,0.0,5000.0,2000.0,0.0,5000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64996,2,3.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,300.0,500.0,6900.0,1400.0,500.0,0.0,20600.0,0.0,200.0,1000.0
64997,2,3.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,26000.0,1900.0,300.0,0.0,32800.0,4500.0,1500.0,1500.0,0.0
64998,1,9.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,23900.0,4300.0,9200.0,2200.0,1300.0,30200.0,1300.0,1300.0,1300.0


### Extract more feture. [ ocp_cd ]

In [21]:
# one hot ocp_cd
demo_f.ocp_cd = pd.to_numeric(demo_f.ocp_cd, downcast='integer')
onehot_ocp = pd.get_dummies(demo_f.ocp_cd, prefix='ocp')
onehot_ocp.head(5)

,ocp_1,ocp_2,ocp_3,ocp_4,ocp_5,ocp_6,ocp_7,ocp_8,ocp_9,ocp_11,ocp_12,ocp_13
id,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0,0,0


In [22]:
demo_f = pd.concat([demo_f, onehot_ocp], axis=1)
demo_f.drop('ocp_cd', axis=1, inplace=True)
demo_f

,gender,age,cc_week_1,cc_week_2,cc_week_3,cc_week_4,cc_week_5,cc_week_6,cc_week_7,cc_week_8,...,ocp_3,ocp_4,ocp_5,ocp_6,ocp_7,ocp_8,ocp_9,ocp_11,ocp_12,ocp_13
id,,,,,,,,,,,,,,,,,,,,,
1,2,5,0.0,0.0,4700.0,0.0,0.0,0.0,20000.0,0.0,...,0,0,0,0,0,0,1,0,0,0
2,2,4,1600.0,0.0,0.0,3500.0,2600.0,800.0,0.0,3800.0,...,1,0,0,0,0,0,0,0,0,0
3,2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
5,2,4,0.0,0.0,5000.0,2000.0,0.0,5000.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64996,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
64997,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
64998,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0


### Extract more feture. [ age ]

In [23]:
onehot_age = pd.get_dummies(demo_f.age, prefix='age')
onehot_age.head()

,age_2,age_3,age_4,age_5,age_6
id,,,,,
1,0,0,0,1,0
2,0,0,1,0,0
3,0,1,0,0,0
4,0,0,0,1,0
5,0,0,1,0,0


In [24]:
demo_f = pd.concat([demo_f, onehot_age], axis=1)
demo_f.drop('age', axis=1, inplace=True)
demo_f

,gender,cc_week_1,cc_week_2,cc_week_3,cc_week_4,cc_week_5,cc_week_6,cc_week_7,cc_week_8,cc_week_9,...,ocp_8,ocp_9,ocp_11,ocp_12,ocp_13,age_2,age_3,age_4,age_5,age_6
id,,,,,,,,,,,,,,,,,,,,,
1,2,0.0,0.0,4700.0,0.0,0.0,0.0,20000.0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
2,2,1600.0,0.0,0.0,3500.0,2600.0,800.0,0.0,3800.0,1200.0,...,0,0,0,0,0,0,0,1,0,0
3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
4,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
5,2,0.0,0.0,5000.0,2000.0,0.0,5000.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64996,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
64997,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
64998,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,1,0,0,0


### Extract more feture. [ gender ]

In [25]:
onehot_gender = pd.get_dummies(demo_f.gender, prefix='gender')
onehot_gender.head()

,gender_1,gender_2
id,,
1,0,1
2,0,1
3,0,1
4,0,1
5,0,1


In [26]:
demo_f = pd.concat([demo_f, onehot_gender], axis=1)
demo_f.drop('gender', axis=1, inplace=True)
demo_f

,cc_week_1,cc_week_2,cc_week_3,cc_week_4,cc_week_5,cc_week_6,cc_week_7,cc_week_8,cc_week_9,cc_week_10,...,ocp_11,ocp_12,ocp_13,age_2,age_3,age_4,age_5,age_6,gender_1,gender_2
id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,4700.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,1
2,1600.0,0.0,0.0,3500.0,2600.0,800.0,0.0,3800.0,1200.0,800.0,...,0,0,0,0,0,1,0,0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,1
5,0.0,0.0,5000.0,2000.0,0.0,5000.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,1
64997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,1
64998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0


## Export data to csv files

In [27]:
demo_f.to_csv('new_finish_data_3.csv', index=False)

----------

# Modelling

In [186]:
df = pd.read_csv('../data_pack/new_finish_data_3.csv')
df.head()

,cc_week_1,cc_week_2,cc_week_3,cc_week_4,cc_week_5,cc_week_6,cc_week_7,cc_week_8,cc_week_9,cc_week_10,...,ocp_11,ocp_12,ocp_13,age_2,age_3,age_4,age_5,age_6,gender_1,gender_2
0,0.0,0.0,4700.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,1
1,1600.0,0.0,0.0,3500.0,2600.0,800.0,0.0,3800.0,1200.0,800.0,...,0,0,0,0,0,1,0,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,1
4,0.0,0.0,5000.0,2000.0,0.0,5000.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1


In [187]:
test = pd.read_csv('../data_pack/test.csv')
test.head()

,id
0,50001
1,50002
2,50003
3,50004
4,50005


In [188]:
train = pd.read_csv('../data_pack/train.csv')
train.head()

,id,income
0,1,20000
1,2,106000
2,3,29000
3,4,61000
4,5,18000


In [189]:
df_f = df[0:50000]
len(df_f)

50000

In [190]:
X_train, X_test, y_train, y_test = train_test_split(df_f, train.income, test_size=0.2)

In [237]:
# read in data
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)
# specify parameters via map
param = {'objective':'reg:squarederror', 'subsample': 0.8, 'min_child_weight': 5, 'max_depth': 9, 'gamma': 2, 'eta': 0.1, 'colsample_bytree': 0.7}
num_round = 11
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

In [238]:
y_test = list(y_test)

In [239]:
sm = 0
for i in range(10000):
    sm += (abs(preds[i] - y_test[i])**2) / ((min(2*abs(y_test[i]), abs(preds[i])) + abs(y_test[i])) ** 2)

In [240]:
score = 100 - 100/10000*sm

In [241]:
score

90.91901706312548

In [242]:
dtest = xgb.DMatrix(df[50000:])
preds = bst.predict(dtest)

In [243]:
dtrain = xgb.DMatrix(df_f, train.income)

In [244]:
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)

In [245]:
test['income'] = preds

## Output

In [300]:
test.to_csv('xg_with_new_data_4.csv', index=False)

In [301]:
df1 = pd.read_csv('xg_with_new_data_90.85.csv')
df2 = pd.read_csv('xg_with_new_data_90.86.csv')
# df3 = pd.read_csv('../XGBoost/xg_v4_lr0.1_91.04.csv')

In [302]:
new_df = pd.concat([df1, df2.income], axis=1)

In [303]:
new_df

,id,income,income
0,50001,22745.736,24149.918
1,50002,25533.900,25148.710
2,50003,30444.105,27232.643
3,50004,23139.557,22316.227
4,50005,34625.620,33378.070
...,...,...,...
14995,64996,18701.133,18731.346
14996,64997,22912.445,25756.139
14997,64998,24148.803,24764.430
14998,64999,21973.047,23783.945


In [304]:
new_df.describe()

,id,income,income
count,15000.000000,15000.000000,15000.000000
mean,57500.500000,31619.069167,31671.230517
std,4330.271354,14875.585247,14935.881633
min,50001.000000,18701.133000,18731.346000
25%,53750.750000,22899.815500,23112.938000
50%,57500.500000,26856.863000,26443.600500
75%,61250.250000,33896.847250,33641.377500
max,65000.000000,172911.830000,160626.020000


In [305]:
test_stack = pd.DataFrame({'id':df1.id, 'income':new_df.median(axis=1)})

In [306]:
test_stack.to_csv('test_stacking2.csv',index=False)